<a href="https://colab.research.google.com/github/Martin-Munive/RAG_MED/blob/main/RAG_MED_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.0. Instalación de librerías.


In [4]:
# --- Core y Modelos de Lenguaje ---
!pip install -q -U google-generativeai #1
!pip install -q -U langchain #2
!pip install -q -U langchain_google_genai #3
# --- Carga y Procesamiento de Documentos ---
!pip install -q -U pypdfium2 #4
# --- Base de Datos Vectorial y Componentes de la Comunidad ---
!pip install -q -U chromadb #5
!pip install -q -U langchain-community #6

## DOCUMENTACIÓN:
# 1. SDK oficial de Google para interactuar con los modelos de IAG.
# 2. Framework principal para flujo del RAG.
# 3. Paquete adaptador que permite a LangChain comunicarse con Gemini.
# 4. Librería base para la manipulación y procesamiento de PDFs.
# -- Permite la extracción del texto.
# 5. Base de datos vectorial. Almacena los "chunks" de texto y sus vectores.
# 6. Paquete de las integraciones y componentes.

## PENDIENTE
# La siguiente librería no es necesaria para el funcionamiento del RAG.
# Se tiene en consideración para una futura optimización.
# Re-ranking de resultados.
# -- Mejora la precisión de la fase de recuperación.
# !pip install -q -U flashrank


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 442.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 933.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 815.6 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 47.7 MB/s eta

## 2.0. Configuración de la API-KEY.

In [5]:
# --- Importación de Módulos Necesarios ---
import google.generativeai as genai #1
from google.colab import userdata #2
import os #3

# DOCUMENTACIÓN
# 1. SDK de Google: para la configuración directa de la API.
# 2. Módulo de Colab: permite acceder a los "Secretos" guardados.
# 3. Módulo del Sistema Operativo: para establecer variables de entorno.

# Configuración Segura de la API Key de Google
# Se gestiona el error durante su verificación.
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada exitosamente desde los secretos de Colab.")
except userdata.SecretNotFoundError:
    print("Error: El secreto 'GOOGLE_API_KEY' no fue encontrado.")
    print("Por favor, sigue las instrucciones para añadir el secreto usando el ícono de la llave (🔑) a la izquierda.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

API Key configurada exitosamente desde los secretos de Colab.


## 3.0. Vinculación de Google Drive.

In [9]:
# --- Importación del Módulo de Drive ---
from google.colab import drive

# --- Ejecución del Montaje ---
drive.mount('/content/drive/')

print("Conectado exitosamente a Google Drive")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Conectado exitosamente a Google Drive


## 4.0. Cargar y dividir los PDFs.

In [10]:
# --- Importación de Módulos para Carga y Procesamiento de Texto ---
import os #1
from langchain_community.document_loaders import PyPDFium2Loader #2
from langchain.text_splitter import RecursiveCharacterTextSplitter #3

# DOCUMENTACIÓN:
# 1. Proporciona funciones para interactuar con el sistema operativo.
# 2. Cargador de directorios de PDF. Escanear la carpeta y buscar archivos.
# 3. Clase para dividir texto. Toma el texto de los PDF y los fragmenta.

# Ruta a la carpeta en Google Drive
DATA_PATH = "/content/drive/MyDrive/RAG_MED-001_DB"
print(f"Buscando archivos PDF en la carpeta: {DATA_PATH}")

# Carga para múltiples PDFs.
all_documents = []

# Itera sobre cada archivo en el directorio.
# Verifica si el archivo es un pdf.
# Se construye la ruta del archivo.
for filename in os.listdir(DATA_PATH):
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(DATA_PATH, filename)
        print(f"Procesando archivo: {file_path}")
        # Se cargan los archivos PDF y se gestiona el error.
        try:
            loader = PyPDFium2Loader(file_path)
            all_documents.extend(loader.load())
        except Exception as e:
            print(f"Error al procesar el archivo {filename}: {e}")


# División de los documentos cargados.
# Se gestiona el error y se renombra la lista.
if not all_documents:
    print("¡Advertencia! No se cargaron documentos. Verifica la ruta y el contenido de la carpeta.")
else:
    documents = all_documents

# Dividimos los documentos en chunks.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    print(f"\nSe han cargado un total de {len(documents)} página(s) desde los archivos PDF.")
    print(f"Se han dividido en {len(docs)} fragmentos (chunks).")
    if docs:
        print("Ejemplo del primer chunk:", docs[0].page_content)

Buscando archivos PDF en la carpeta: /content/drive/MyDrive/RAG_MED-001_DB
Procesando archivo: /content/drive/MyDrive/RAG_MED-001_DB/Gross-Anatomy-The-Big-Picture.pdf


/usr/local/lib/python3.11/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")



Se han cargado un total de 513 página(s) desde los archivos PDF.
Se han dividido en 2844 fragmentos (chunks).
Ejemplo del primer chunk: GROSS ANATOMY
THE BIG PICTURE


## 4.1. Verificación de los chunks.



In [11]:
# Imprime los primeros 5 chunks para inspeccionar su calidad.
print("--- Verificando la calidad de los primeros 5 chunks ---")
for i, doc in enumerate(docs[:5]):
    print(f"--- CHUNK {i+1} (Página: {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content)
    print("---------------------------------------------------\n")
# Muestra exactamente cómo se está dividiendo el PDF.
# Si hay títulos de sección, texto desordenado o contenido sin sentido.
# --- Permite identificar problemas es la extracción del PDF.


--- Verificando la calidad de los primeros 5 chunks ---
--- CHUNK 1 (Página: 1) ---
GROSS ANATOMY
THE BIG PICTURE
---------------------------------------------------

--- CHUNK 2 (Página: 2) ---
Notice
Medicine is an ever-changing science. As new research and clinical experience broaden our knowledge, changes in treatment and drug therapy are
required. The authors and the publisher of this work have checked with sources believed to be reliable in their efforts to provide information that
is complete and generally in accord with the standards accepted at the time of publication. However, in view of the possibility of human error or
---------------------------------------------------

--- CHUNK 3 (Página: 2) ---
changes in medical sciences, neither the authors nor the publisher nor any other party who has been involved in the preparation or publication
of this work warrants that the information contained herein is in every respect accurate or complete, and they disclaim all responsibilit

## 5.0. Se vincula la Base de datos.

In [13]:
# --- Importación de Módulos para Vectorización y Almacenamiento ---
from langchain_google_genai import GoogleGenerativeAIEmbeddings #1
from langchain_community.vectorstores import Chroma #2

# DOCUMENTACIÓN.
# 1. Actua como 'traductor semántico':
# -- Convierte los fragmentos de texto ('chunks') en números (un vector).
# 2. Clase que LangChain utiliza para comunicarse con la DB.

# 1. Definir el modelo de embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 2. Crear la base de datos vectorial con los chunks y sus embeddings.
# Esta línea tomará cada 'chunk' de 'docs', lo convertirá en un vector.
# -- Usando el modelo de embeddings,
# -- Lo almacenará en ChromaDB.
vector_store = Chroma.from_documents(docs, embeddings, persist_directory="chroma_db")

print("Base de datos vectorial creada y guardada en el directorio 'Chroma_DB'.")

Base de datos vectorial creada y guardada en el directorio 'chroma_db'.


## 6.0. Definir el LLM.

In [15]:
# --- Importación del Módulo del Modelo de Lenguaje (LLM) ---
from langchain_google_genai import ChatGoogleGenerativeAI

# Se elige el modelo.
# Se define una temperatura no muy alta. Respuestas precisas, no muy creativas.
LLM = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest",
                         temperature=0.8)

print("LLM configurado exitosamente con el modelo: Gemini-1.5-flash-latest.")

LLM configurado exitosamente con el modelo: Gemini-1.5-flash-latest.


## 7.0. Entorno de consulta.

In [16]:
# --- Importación de Componentes para la Construcción de la Cadena RAG ---
from langchain.chains import create_retrieval_chain #1
from langchain.chains.combine_documents import create_stuff_documents_chain #2
from langchain_core.prompts import ChatPromptTemplate #3

# DOCUMENTACIÓN:
# 1. Función para crear la cadena de recuperación.
# -- Tomar la pregunta del usuario y usar el 'retriever' para buscar el contexto.
# -- Pasa la pregunta y el contexto recuperado a la siguiente cadena en la secuencia.
# -- (la 'document_chain') para que genere la respuesta final.
# 2. Función específica para manejar documentos.
# -- Toma todos los fragmentos recuperados, los junta en un solo bloque.
# -- los inserta en la plantilla del prompt junto con la pregunta del usuario.
# 3. Clase para crear plantillas de prompts.

# "RETRIEVER": busca en la base de datos vectorial.
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 20}
    # 'fetch_k' busca z chunks, 'k' selecciona los x mejores y más diversos.
)

# Prompt para dar las instrucciones al LLM.
# SOLO usará el contexto (los chunks) para responder.
# Este primer ejemplo se realizó con documentos sobre Anatomía.
prompt = ChatPromptTemplate.from_template("""
Actúa como un asistente experto en anatomía humana. Tu tarea es responder la pregunta del usuario. Para ello, analiza y sintetiza la información de TODOS los fragmentos proporcionados en el siguiente contexto.

<context>
{context}
</context>

Utiliza la información del contexto para construir una respuesta detallada y completa a esta pregunta: {input}

Presta especial atención a detalles específicos como regiones anatómicas, inervaciones, irrigaciones, contenido de los espacios y relaciones espaciales. Si varios fragmentos se complementan, combínalos en una única respuesta coherente.

Si, después de analizar todo el contexto, la información necesaria no se encuentra, responde de forma clara y directa: "La información solicitada no se encuentra en los documentos proporcionados."
""")

# Combina los documentos recuperados en un solo bloque de texto.
document_chain = create_stuff_documents_chain(LLM, prompt)

# Une todo: recupera documentos y luego genera la respuesta.
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("Cadena RAG creada exitosamente.")

Cadena RAG creada exitosamente.


## 8.0. Hacer preguntas.

In [17]:
pregunta = "What is the subarachnoid space?"

# Invocamos la cadena con nuestra pregunta.
response = retrieval_chain.invoke({"input": pregunta})

# Imprimimos la respuesta
print("Respuesta generada:")
print(response["answer"])

# Para ver qué chunks usó para responder
print("\n--- Contexto Utilizado ---")
for doc in response["context"]:
    print(f"Fuente: {doc.metadata.get('source', 'N/A')}, Página: {doc.metadata.get('page', 'N/A')}")
    print(doc.page_content)
    print("--------------------------")

Respuesta generada:
El espacio subaracnoideo es la cavidad ubicada entre la aracnoides y la piamadre, dos de las tres meninges que protegen al encéfalo y la médula espinal.  Contiene líquido cefalorraquídeo (LCR), el cual suspende la médula espinal, el cerebro y las raíces nerviosas.  Además, grandes vasos sanguíneos recorren el espacio subaracnoideo.  Una parte del espacio subaracnoideo se conoce como saco dural.  La aracnoides, la capa meníngea intermedia, está unida a la piamadre subyacente por numerosas trabéculas aracnoideas.

--- Contexto Utilizado ---
Fuente: /content/drive/MyDrive/RAG_MED-001_DB/Gross-Anatomy-The-Big-Picture.pdf, Página: 192
period of days or even a week. Enlarging the subdural space is one
factor that increases the risk of a subdural hematoma. As the subdural space enlarges, the bridging veins that traverse the space
travel over a wider distance, causing them to be more vulnerable
to tears. As a result, infants (who have smaller brains), the elderly
(whose br

## 9. Listar modelos de LLM disponibles.

In [18]:
# OPCIONAL:
# Verificación de Modelos de IA Disponibles.
import google.generativeai as genai #1

#1. Permite consultar directamente a la API de Google para obtener una lista
#   de todos los modelos de IAG a los que la API Key tiene acceso.

for model in genai.list_models():
  if 'generateContent' in model.supported_generation_methods:
    print(model.name)

models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp